## RLVR Example - Finetuning with Sagemaker

This notebook demonstrates basic user flow for RLVR Finetuning from a model available in Sagemaker Jumpstart.
Information on available models on jumpstart: https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-latest.html

### Setup and Configuration

Initialize the environment by importing necessary libraries and configuring AWS credentials

In [1]:

from sagemaker.train.rlvr_trainer import RLVRTrainer
from sagemaker.train.configs import InputData
from rich import print as rprint
from rich.pretty import pprint
from sagemaker.core.resources import ModelPackage
from sagemaker.train.common import TrainingType


import boto3
from sagemaker.core.helper.session_helper import Session
import os
os.environ['SAGEMAKER_REGION'] = 'us-east-1'
os.environ['SAGEMAKER_STAGE'] = 'prod'

# Beta endpoint configuration
api_endpoint = "https://api.sagemaker.gamma.us-west-2.ml-platform.aws.a2z.com"

# Create SageMaker clients with beta endpoints
sm_client = boto3.client('sagemaker', endpoint_url=api_endpoint, region_name="us-west-2")

# Create SageMaker session with beta endpoints
sagemaker_session = Session(sagemaker_client=sm_client)




[11/24/25 14:23:24] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=378592;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=654878;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/rsareddy/Library/Application Support/sagemaker/config.yaml


                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=26354;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=186501;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

### Create RLVRTrainer
**Required Parameters** 

* `model`: base_model id on Sagemaker Hubcontent that is available to finetune (or) ModelPackage artifacts

**Optional Parameters**
* `custom_reward_function`: Custom reward function/Evaluator ARN
* `model_package_group_name`: ModelPackage group name or ModelPackageGroup
* `mlflow_resource_arn`: MLFlow app ARN to track the training job
* `mlflow_experiment_name`: MLFlow app experiment name(str)
* `mlflow_run_name`: MLFlow app run name(str)
* `training_dataset`: Training Dataset - either Dataset ARN or S3 Path of the dataset (Please note these are required for a training job to run, can be either provided via Trainer or .train())
* `validation_dataset`: Validation Dataset - either Dataset ARN or S3 Path of the dataset
* `s3_output_path`: S3 path for the trained model artifacts

In [8]:
# For fine-tuning 
rlvr_trainer = RLVRTrainer(
    model="meta-textgeneration-llama-3-2-3b-instruct", # Union[str, ModelPackage] 
    model_package_group_name="test-finetuned-models-gamma", #"test-finetuned-models", # Make it Optional
    #mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-rlvr-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-rlvr-finetuned-models-run", # Optional[str]
    training_dataset="s3://rsareddy-test-for-demo/input_data/rlvr-rlaif-oss-dataset/train_285.jsonl",     #"arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/MarketingDemoDataset1/1.0.0", #Optional[]
    s3_output_path="s3://rsareddy-test-for-demo/output/",
    sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/service-role/AmazonSageMaker-ExecutionRole-20250731T162975"
    role="arn:aws:iam::052150106756:role/Admin"
)

### Discover and update Finetuning options

Each of the technique and model has overridable hyperparameters that can be finetuned by the user.

In [9]:
print("Default Finetuning Options:")
pprint(rlvr_trainer.hyperparameters.to_dict()) # rename as hyperparameters

#set options
rlvr_trainer.hyperparameters.get_info()



Default Finetuning Options:


{
│   'data_path': 'None',
│   'global_batch_size': '256',
│   'learning_rate': '3e-05',
│   'lora_alpha': '32',
│   'max_epochs': '1',
│   'max_prompt_length': '1024',
│   'mlflow_run_id': '',
│   'mlflow_tracking_uri': '',
│   'model_name_or_path': 'meta-llama/Llama-3.2-3B-Instruct',
│   'name': 'example-name-c9tcy',
│   'output_path': '/opt/ml/model',
│   'preset_reward_function': '',
│   'results_directory': '',
│   'resume_from_path': '',
│   'reward_lambda_arn': '',
│   'rollout': '6',
│   'train_val_split_ratio': '0.9',
│   'validation_data_path': 'None'
}


data_path:
  Current value: None
  Type: string
  Default: None
  Required: Yes

global_batch_size:
  Current value: 256
  Type: integer
  Default: 256
  Valid options: [256, 512, 1024]
  Required: Yes

learning_rate:
  Current value: 3e-05
  Type: float
  Default: 3e-05
  Range: 1e-07 - 0.001
  Required: Yes

lora_alpha:
  Current value: 32
  Type: integer
  Default: 32
  Range: 8 - 1024
  Required: Yes

max_epochs:
  Current value: 1
  Type: integer
  Default: 1
  Range: 1 - 100000
  Required: Yes

max_prompt_length:
  Current value: 1024
  Type: integer
  Default: 1024
  Range: 512 - 16384
  Required: Yes

mlflow_run_id:
  Current value: 
  Type: string
  Default: 

mlflow_tracking_uri:
  Current value: 
  Type: string
  Default: 

model_name_or_path:
  Current value: meta-llama/Llama-3.2-3B-Instruct
  Type: string
  Default: meta-llama/Llama-3.2-3B-Instruct
  Required: Yes

name:
  Current value: example-name-c9tcy
  Type: string
  Default: example-name-c9tcy
  Required: Yes

outp

#### Start RLVR training


In [4]:
training_job = rlvr_trainer.train(wait=True)

[11/24/25 13:51:00] INFO     Training Job Name:                                                 ]8;id=742411;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/rlvr_trainer.py\rlvr_trainer.py]8;;\:]8;id=337691;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/rlvr_trainer.py#111\111]8;;\
                             meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124135100                         

Training Job Name: meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124135100


                    INFO     MLflow resource ARN:                                             ]8;id=717930;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=671099;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#436\436]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/                      
                             mmlu-eval-experiment                                                                  

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/rsareddy/Library/Application Support/sagemaker/config.yaml


                    INFO     Creating training_job resource.                                     ]8;id=555673;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=164442;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35539\35539]8;;\

Output()

[11/24/25 13:59:12] INFO     Final Resource Status: Completed                                    ]8;id=82199;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=719590;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35872\35872]8;;\

In [10]:
training_job = rlvr_trainer.train(wait=False)

[11/24/25 14:30:11] INFO     Training Job Name:                                                 ]8;id=822059;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/rlvr_trainer.py\rlvr_trainer.py]8;;\:]8;id=127182;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/rlvr_trainer.py#111\111]8;;\
                             meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124143011                         

[11/24/25 14:30:14] INFO     Using first available MLflow app:                                 ]8;id=257174;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=961869;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#72\72]8;;\
                             arn:aws:sagemaker:us-east-1:052150106756:mlflow-app/app-5B5KAOX4I                     
                             UO7                                                                                   

                    INFO     MLflow resource ARN:                                             ]8;id=781137;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=178116;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#436\436]8;;\
                             arn:aws:sagemaker:us-east-1:052150106756:mlflow-app/app-5B5KAOX4                      
                             IUO7                                                                                  

                    INFO     Creating training_job resource.                                     ]8;id=380631;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=674077;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35539\35539]8;;\

### View any Training job details

We can get any training job details and its status with TrainingJob.get(...)

In [6]:
from sagemaker.core.resources import TrainingJob

response = TrainingJob.get(training_job_name="meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123033517")
pprint(response)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123033517',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123033517',
│   processing_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   tuning_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   labeling_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   auto_ml_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123033517/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123033517/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '256',
│   │   'learning_rate': '3e-05',
│   │   'lora_alpha': '32',
│   │   'max_epochs': '1',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-3B-Instruct',
│   │   'name': 'example-name-5k253',
│   │   'output_path': '/opt/ml/model',
│   │   'rollout': '6',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   │   file_system_data_source=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   shuffle_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   keep_alive_period_in_seconds=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   capacity_reservation_ids=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_groups=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   capacity_schedules_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   training_plan_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_placement_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>
│   ),
│   warm_pool_status=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   vpc_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_i

In [14]:
training_job.refresh()
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124143011',
│   training_job_arn='arn:aws:sagemaker:us-east-1:052150106756:training-job/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124143011',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://rsareddy-test-for-demo/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124143011/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://rsareddy-test-for-demo/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124143011/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '256',
│   │   'learning_rate': '3e-05',
│   │   'lora_alpha': '32',
│   │   'max_epochs': '1',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-3B-Instruct',
│   │   'name': 'example-name-c9tcy',
│   │   'output_path': '/opt/ml/model',
│   │   'rollout': '6',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=S3DataSource(
│   │   │   │   │   s3_data_type='S3Prefix',
│   │   │   │   │   s3_uri='s3://rsareddy-test-for-demo/input_data/rlvr-rlaif-oss-dataset/train_285.jsonl',
│   │   │   │   │   s3_data_distribution_type='FullyReplicated',
│   │   │   │   │   attribute_names=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   │   │   instance_group_names=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   │   │   model_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   │   │   hub_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>
│   │   │   │   ),
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   │   dataset_source=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://rsareddy-test-for-demo/output/',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   keep_alive_period_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   capacity_reservation_ids=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   instance_groups=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │  

### Test RLVR with Custom RewardFunction

Here we are providing a user-defined reward function ARN

In [3]:

# For fine-tuning 
rlvr_trainer = RLVRTrainer(
    model="meta-textgeneration-llama-3-2-1b-instruct", # Union[str, ModelPackage] 
    model_package_group_name="test-finetuned-models-gamma", # Make it Optional
    mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-rlvr-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-rlvr-finetuned-models-run", # Optional[str]
    training_dataset="arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2", #Optional[]
    s3_output_path="s3://open-models-testing-pdx/output",
    sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/service-role/AmazonSageMaker-ExecutionRole-20250731T162975"
    role="arn:aws:iam::052150106756:role/Admin",
    custom_reward_function="arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/JsonDoc/rlvr-test-rf/0.0.1"
    
)

[11/19/25 22:03:42] WARNING  No config provided. Using default config.                                 ]8;id=604233;file:///Users/rsareddy/.local/share/mise/installs/python/3.11.9/lib/python3.11/site-packages/sagemaker_core/main/utils.py\utils.py]8;;\:]8;id=154575;file:///Users/rsareddy/.local/share/mise/installs/python/3.11.9/lib/python3.11/site-packages/sagemaker_core/main/utils.py#347\347]8;;\

In [4]:
training_job = rlvr_trainer.train(wait=True, logs=True)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251119220405     │
│  MLFlow URL            ]8;id=684727;https://t-eq86xu3xab1s.us-west-2.experiments.sagemaker.aws/auth?authToken=eyJhbGciOiJIUzI1NiJ9.eyJhdXRoVG9rZW5JZCI6IjQzOGJiZDNjLWVmZjItNDI4OS04ZDE3LWMwNTYzMmE1ZmNjMiIsImZhc0NyZWRlbnRpYWxzIjoiQWdWNC81SnZSVjNPb1pKOUVZU3hreHBIL3J5T054ZXVFS0lsdEVtd0pkaXQyRkVBWHdBQkFCVmhkM010WTNKNWNIUnZMWEIxWW14cFl5MXJaWGtBUkVFclJVcHdlblF5Y2xCTWRITXlWazlNUld0elZYZE5hekI1TmpkdFQxZ3JSMjB6UjBOb1RqWjBjVkJEYjI5SFFWSnNZMlJWUTI1d2MyaElWRFI1VlVWMWR6MDlBQUVBQjJGM2N5MXJiWE1BUzJGeWJqcGhkM002YTIxek9uVnpMWGRsYzNRdE1qbzFPVEF4T0RNM016azFNRFE2YTJWNUx6ZzNabUUxTVdReUxURTRNRGt0TkdVMFl5MWhObVV6TFRRNFpXWTNNelk1WW1NM1lnQzRBUUlCQUhndFR2ZzEyNVZtTm50WE4vTEVZV1dzREMrSk1KRzNQSUl1eU9PYTB5SU5FZ0dqY0k0dHhRVHFPQndRU3hBQU9kaWZBQUFBZmpCOEJna3Foa2lHOXcwQkJ3YWdiekJ0QWdFQU1HZ0dDU3FHU0liM0RRRUhBVEFlQmdsZ2hrZ0JaUU1FQVM0d0VRUU16ZHJFVGtIWUNtdGp3V1E5QWdFUWdEdGZTcTZvRXdLL3UvMGNpa1AzWE0vUUJnb1grYUFOZ2hRSUY4U2VrcTVYcGZyaFUwTGIvRmhjVEFLczFpZ09xOG0yQUg5UTVQS3RvdXdnUkFJQUFCQUFVdGxJNVdQc09pbjlXUTVhbW8vRzE3VklacU9rTkJycDlKQnFtZWNRbzVwZWpUakNmQnpJR2RrQVM3WHFvNkRqLy8vLy93QUFBQUVBQUFBQUFBQUFBQUFBQUFFQUFBUTdrRFd0TWJ5UlJ2SmFsaXFLd1creUlMNkRUbE82cXI4b01VaEZpdVVXN1JhKzQ3QWduRkM5all5Y2pRN1Y0MjU2VzdJdkYvSThnSk83Z0d1M0pqVXppeUNoSnZVajFpUUMzUGgrSkpkNzJIRFlNL2Zrd1hncjZBZlVvRTFGeGwxc1JPMXRiZ0ErVXRVemNoNldGWlVJQUtrVDZ3cHdTZ0NKZDl0Z1Z0UVhidlFBVGk5Nkd4TXhwNVpnTXQyYlV3Yi8zTUJGMkxDdWxRWWxuMVVLL3k2RndkdGVZVmc2MUNqMkJPczRkY0F4enBPbWMyQ1lpQnJacDZQcWYxWVlNcUN0TmdycVlYdjJoMDZsNFk2a0VWRWRsUHRxQlR2Y3lTTVBnVEhRMytLWjVWUitaWHZGN2s1WnJEc2h3Q283YkZ6Mk9TWHpCMHdiTzRtTjA0M1BVMnJWOU9KbGxjYmZ1WUE0eUgyd29aY0hwUUtxQkxPNXkyT3gvdmF6UGVIM0lRdVNnM1VtK3RWMWVxUjZTUzYwTGI0U3Y4bHZtdHV3ZzBSa20wMFhhYUh3V2lMVGR3dk16OURGNE9hZ012dTBDQUduR1pOVitLV2FZcnE1c05ubk1ZdGZFRlZHOHJWamVsQytVbXlQRzBlT0xZT0FKNkVCQ1JwYkRaYjl6Tmg4QTdnYXRtSkVHYnZ4Mk5KMjRZTUwwR2pBOE4wSVBVdHpDZE9wVEtKcldSaTN1YnBQOURMRkVZMWxseE9DTGRxU2U4T29TTzNhQTlPTDZVaDFoSHlQSi8ycmR4ZjFhZXdTeXk5SVMxc3NTN21nWjQyY1grZUs2MkFLeUdQckhVRDQ5V0ZLMnEyYVdhdFBNSFFDKy9VM2JVME44Z1F1SlpleVRGK2RGMHB5dmZVR2YwNXRvd0NLVjFEcGFnRGt6Y2Q3Z21taU5yVFRmb3dISXlobFFPZVpOVGJUKzR5dzRDTHE0THg0RndCU2VmekM2aFBSN2JqdGI2M1lpdHhPeFFJdDZsNG9rM0piTENURE40VlhDc2wzbDg0ZmtRQUpjZXlSbklBYk4xV1VMVlByekRLS3RXM1oyM2hiU21ibmRFeXJIekdINW1nUjU2a0t5UE00cVZCc1V2eWEyMkVNS283NjJjZjUwZloxYXpHZ3BGOCtLSmVpNlRhL0RLS1FkZzFITlZjeW9HeXlBMnE0RWNnOE81S3dubHU3YktjWlJwbWE5NnBiRFVoWG5YWlhKVUY4SHMrTkdmcENDRXFRbkFQVFM0Ylo1ai9WWWVTdG5FSDgxS1RCZHNPQXhXbjgvQmJNcDlyRmc4WnBPL1ZHODBESDVTYWcrZElWZkE1UlE1eVpMRWxEU2dQV2J6cWRtcUt0MmVyampPUnlFcUY0S1JXMW4zWGo4VTZHWElxWlROUFBPWDE2cFp0T2l0NkE5TXExVEJzRnFHMlkydUdVRTU1RTVMWGk1aDhBRHY4MFNWR1hiRzNheTNxRDRBMjdqcWQ3aE56aTRlVTF5QURCODV4NElKcTJ6UmtEblo5aVZvVGJwZ0NlN2VBc25BbkdLYUw0TTdvVmc3WkZRUko4cFB4T01yV09WSnNCOUo1OGVtUXdHTWkzMUVqWHF4RWgvbTV3WXh6Z3I2dGlYQUo0WWw5NlhWZVhoUm5CZTBsb1hqUitkdEtEQ0VzZkZIbjUzYytzVTVWcHMrNkdWMnUzNmhNK3Y3OXo5L0FMQjlia1BESXYvYlhmTUk2YVcxbnNRekxUMWc5LzYvcGN4WjFSdE5udUlsWDZvSXdYbHBnQXdkSVFVdmJRcXFremo1RmgxeFNlbUU0dmN5ZGRhK3dCdTBITGJCdTlDZEw5ZHVqV05jWHpsTy9Sd0szekhUZTBUbEt4bkxER0dhaTZid0hsSTV5SkpDczMrMTlEWEhHcVd3ZFNaTGJGbWpSS0RnK1UyZ0JuTUdVQ01RRGkyTis1aWhzcmVhMm02VlB2ei9BalVOWXF1YUxJejh6NUlIb0dVaTZ2K1BVNnc0U1lkYklQZmRRcnlMVTBMWGdDTUdTVmtSOEJFVEw1VisvSDhUMXV6akpxWEdFZ2l1SVc4TUd3bFdRbCtqWEdpUG4zaG9TTkhOU2pwK0J5VXMvM3BBPT0iLCJjaXBoZXJUZXh0IjoiQVFJQkFIZ3RUdmcxMjVWbU5udFhOL0xFWVdXc0RDK0pNSkczUElJdXlPT2EweUlORWdIZHZyY0RuRGtONDFFVlBRbW9XUEkwQUFBQW9qQ0Jud1lKS29aSWh2Y05BUWNHb0lHUk1JR09BZ0VBTUlHSUJna3Foa2lHOXcwQkJ3RXdIZ1lKWUlaSUFXVURCQUV1TUJFRURJVmZnV1VvNzFQM2tISGg0d0lCRUlCYlVxelJjVG41TXp5Ty80ZFJPRzdJRkhHSzVpbVQrWmFiMVMyR0JyVVdpMFZyTU5ubDJEMTBnSExnZVpPNHB5a01OVURVbkZSalE4cVUvTWgvR2lheHc4REtpZUoyaFVZdkd2U1Q4ajY2MGczVkcyb2RYM21KNHRaa2NnPT0iLCJzdWIiOiJhcm46YXdzOnNhZ2VtYWtlcjp1cy13ZXN0LTI6MDUyMTUwMTA2NzU2Om1sZmxvdy10cmFja2luZy1zZXJ2ZXIvbW1sdS1ldmFsLWV4cGVyaW1lbnQiLCJpYXQiOjE3NjM2MTg2NDYsImV4cCI6MTc2MzYx

In [10]:
#training_job.refresh()
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251118231012',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251118231012',
│   processing_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   tuning_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   labeling_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   auto_ml_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251118231012/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251118231012/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '256',
│   │   'learning_rate': '3e-05',
│   │   'lora_alpha': '32',
│   │   'lora_rank': '32',
│   │   'max_epochs': '1',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-itz74',
│   │   'output_path': '/opt/ml/model',
│   │   'reward_lambda_arn': 'arn:aws:lambda:us-west-2:052150106756:function:sm-eval-vinayshm-rlvr-llama-321b-instruct-v1-1762713051528',
│   │   'rollout': '6',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   │   │   file_system_data_source=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   │   shuffle_config=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>
│   ),
│   resource_config=ResourceConfig(
│   │   volume_size_in_gb=0,
│   │   instance_type=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   instance_count=0,
│   │   volume_kms_key_id=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   keep_alive_period_in_seconds=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   capacity_reservation_ids=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   instance_groups=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   capacity_schedules_config=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   training_plan_arn=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>,
│   │   instance_placement_config=<sagemaker_core.main.utils.Unassigned object at 0x108ea4ed0>
│   ),
│   warm_pool_status=<sagemaker_core.main.utils.Unassigned

In [ ]:

#meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251113182932

## Continued Finetuning (or) Finetuning on Model Artifacts

#### Discover a ModelPackage and get its details

In [6]:
from rich import print as rprint
from rich.pretty import pprint
from sagemaker.core.resources import ModelPackage, ModelPackageGroup

#model_package_iter = ModelPackage.get_all(model_package_group_name="test-finetuned-models-gamma")
model_package = ModelPackage.get(model_package_name="arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/61")

pprint(model_package)

ModelPackage(
│   model_package_name=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   model_package_group_name='test-finetuned-models-gamma',
│   model_package_version=61,
│   model_package_registration_type='Logged',
│   model_package_arn='arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/61',
│   model_package_description=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   creation_time=datetime.datetime(2025, 11, 23, 3, 44, 26, tzinfo=tzlocal()),
│   inference_specification=InferenceSpecification(
│   │   containers=[
│   │   │   ContainersItem(
│   │   │   │   model_data_url=None,
│   │   │   │   image=None,
│   │   │   │   nearest_model_name=None,
│   │   │   │   model_data_source=ModelDataSource(),
│   │   │   │   is_checkpoint=False,
│   │   │   │   base_model=BaseModelInfo(hub_content_name='meta-textgeneration-llama-3-2-3b-instruct')
│   │   │   )
│   │   ]
│   ),
│   source_algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   validation_specification=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   model_package_status='Completed',
│   model_package_status_details=ModelPackageStatusDetails(validation_statuses=[], image_scan_statuses=[]),
│   certify_for_marketplace=False,
│   model_approval_status=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   created_by=UserContext(
│   │   user_profile_arn=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   │   user_profile_name=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   │   domain_id=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   │   iam_identity=IamIdentity(
│   │   │   arn='arn:aws:sts::052150106756:assumed-role/Admin/SageMaker',
│   │   │   principal_id='AROAQYJDDPKCM5Q7HXZYK:SageMaker',
│   │   │   source_identity=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>
│   │   )
│   ),
│   metadata_properties=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   model_metrics=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   deployment_specification=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   last_modified_time=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   last_modified_by=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   approval_description=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   domain=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   task=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   sample_payload_url=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   sample_payload_content_type=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   customer_metadata_properties=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   drift_check_baselines=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   additional_inference_specifications=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   skip_model_validation=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   source_uri=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   security_config=ModelPackageSecurityConfig(
│   │   kms_key_id=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>
│   ),
│   model_card=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   model_life_cycle=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>
)

#### Create Trainer

Trainer creation is same as above Finetuning Section except for `model`'s input is ModelPackage(previously trained artifacts)

In [7]:
# For fine-tuning 
rlvr_trainer = RLVRTrainer(
    model=model_package, # Union[str, ModelPackage] 
    training_type=TrainingType.LORA, 
    model_package_group_name="test-finetuned-models-gamma", #"test-finetuned-models", # Make it Optional
    mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-rlvr-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-rlvr-finetuned-models-run", # Optional[str]
    training_dataset="arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2",     #"arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/MarketingDemoDataset1/1.0.0", #Optional[]
    s3_output_path="s3://open-models-testing-pdx/output",
    sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/service-role/AmazonSageMaker-ExecutionRole-20250731T162975"
    role="arn:aws:iam::052150106756:role/Admin"
)

In [2]:
# For fine-tuning 
rlvr_trainer = RLVRTrainer(
    model="arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/61", # Union[str, ModelPackage] 
    training_type=TrainingType.LORA, 
    model_package_group_name="test-finetuned-models-gamma", #"test-finetuned-models", # Make it Optional
    mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-rlvr-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-rlvr-finetuned-models-run", # Optional[str]
    training_dataset="arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2",     #"arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/MarketingDemoDataset1/1.0.0", #Optional[]
    s3_output_path="s3://open-models-testing-pdx/output",
    sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/service-role/AmazonSageMaker-ExecutionRole-20250731T162975"
    role="arn:aws:iam::052150106756:role/Admin"
)

[11/23/25 06:49:59] WARNING  No config provided. Using default config.                                 ]8;id=231674;file:///Users/rsareddy/.local/share/mise/installs/python/3.11.9/lib/python3.11/site-packages/sagemaker_core/main/utils.py\utils.py]8;;\:]8;id=264143;file:///Users/rsareddy/.local/share/mise/installs/python/3.11.9/lib/python3.11/site-packages/sagemaker_core/main/utils.py#347\347]8;;\

us-west-2


#### Start the Training

In [ ]:
training_job = rlvr_trainer.train(
    wait=True,
)

[11/24/25 14:01:03] INFO     Training Job Name:                                                 ]8;id=355721;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/rlvr_trainer.py\rlvr_trainer.py]8;;\:]8;id=369327;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/rlvr_trainer.py#111\111]8;;\
                             meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124140103                         

Training Job Name: meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124140103


                    INFO     MLflow resource ARN:                                             ]8;id=100728;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=480551;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#436\436]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/                      
                             mmlu-eval-experiment                                                                  

                    INFO     Creating training_job resource.                                     ]8;id=436922;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=176368;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35539\35539]8;;\

Output()

In [11]:
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123035854',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123035854',
│   processing_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   tuning_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   labeling_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   auto_ml_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123035854/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123035854/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '256',
│   │   'learning_rate': '3e-05',
│   │   'lora_alpha': '32',
│   │   'max_epochs': '1',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-3B-Instruct',
│   │   'name': 'example-name-5k253',
│   │   'output_path': '/opt/ml/model',
│   │   'rollout': '6',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   │   file_system_data_source=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   shuffle_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   keep_alive_period_in_seconds=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   capacity_reservation_ids=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_groups=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   capacity_schedules_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   training_plan_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_placement_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>
│   ),
│   warm_pool_status=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   vpc_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_i